## Урок 3.  Домашнее задание.

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

import os, sys
sys.path.insert(1, os.getcwd() + '/webinar_3')
from metrics import precision_at_k, recall_at_k

In [98]:
data = pd.read_csv('webinar_3/data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


__Задание 1.__ <br> **Ситуация**: Вы работаете data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи. <br>
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- __Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)__ <br>
_Сразу оговорюсь, что не работал дата сайентистом, поэтому мои предположения могут быть неправильными. Я бы ожидал, что менеджер хочет от рекомендательной системы увеличения общей выручки/прибыли или выручки/прибыли с 1 клиента. Наверное он хочет, чтобы рост продаж был выше того, который наблюдался у конкурента или среднего по отрасли. Сложно сказать, за какое количество недель он хочет получить такой рост, однако опять же в первую очередь, на мой взгляд, сравнение будет идти с конкурентом или средним по отрасли. Значит, построенная рекомендательная система должна быть лучше, чем указанные величины._ <br><br>
- __По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)__ <br>
_Согласен с этим утверждением, это необходимо, чтобы понимать нужно ли тратить время и ресурсы на построение такой системы, или лучших результатов с учетом существующих ожиданий можно добиться каким-то более простым/быстрым путем._ <br><br>
- __А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?__ <br>
_Ответ на этот вопрос косвенно связан с предыдущим, так как зачастую дата сайентисты лучше представляют то, как хранятся данные в компании, насколько они актуальны, и как легко их извлечь для построения каких-либо моделей/рекомендательных систем. Если e-mailы пользователей не были собраны ранее, то этот процесс может занять определенное время и тоже потребует ресурсов, которые фактически будут означать издержки для планируемой рекомендательной системы. Аналогично с актуализацией базы данных._ <br><br>
- __Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?__ <br>
_Здесь, на мой взгляд, с одной стороны играют роль издержки каждого из подхода: что будет стоить дешевле - СМС/push на телефон или печать дополнительной информации на чеке. С другой стороны, и это по моему мнению может иметь больше значение в данном случае, вовлечение клиента в каждом из данных случаев. Для этого нужно понимание нашей аудитории и в целом стратегии компании. Если например, основную часть клиентов составляют пожилые люди, то СМС/push на телефон будет явно менее эффективен, чем печать рекомендаций на чеке. Аналогично, если мы являемся дискаунтером, в котором большое количество покупок совершается по акции, то размещение рекомендаций на чеке может иметь смысл, так как есть категория людей, которые проверяют цены в чеке после оплаты на кассе, соответственно они с большей вероятностью увидят наши рекомендации/предложения. Если же мы позиционируем себя как максимально экологичный магазин, который делает упор на приложении и призывает отказаться от печати чеков, то размещение рекомендаций на нем будет бесмысленно._ <br><br>
- __Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)__ <br>
_Здесь наверное нужны дополнительные консультации с людьми, отвечающими за e-mail рассылки и коммуникации с пользователями, чтобы понимать, каким образом должен выглять e-mail, чтобы пользователь совершил необходимое действие._ <br><br>
- __Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?__ <br>
_Логично чтобы в рекомендациях присутствовали не только акционные товары, так как среди прочего мы наверняка хотим увеличить продажи в денежном выражении или прибыль, поэтому в рекомендации точно должны попадать среди прочего высокомаржинальные товары. Можно включить также сезонные товары._ <br><br>
- __Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки__ <br>
_Ответ на этот вопрос наверное лучше всего представляет менеджмент, плюс необходимы какие-то среднеотраслевые показатели, который могут служить точкой отсчета. С нашей точки зрения нужно понимание по издержкам в зависимости от параметров кампании (каналы коммуникации, количество пользователей на которых ориентирована) для того, чтобы понимать насколько она будет реализуема в том виде, в котором хочет менеджмент._ <br><br>
- __Cколько мы хотим зарабатывать с одного привлеченного юзера?__ <br>
_Предполагаю, что в долгосрочной перспективе прибыль с одного клиента, привлеченного таким образом, не должна быть меньше прибыли с обычной покупки. Также важно понимать, какую долю в общем объеме продаж занимают продажи, осуществленные после получения рекомендаций._ <br>

---
- __А точно нужно сортировать по вероятности?__ <br>
_Во-первых нужно учесть ограничения, отмеченные выше, касающиеся типов предлагаемых товаров. Во-вторых, можно сортировать не только по простой вероятности, но и учесть нашу маржинальность и сортировать таким образом по ожидаемой маржинальности._ <br> <br>
- __Какую метрику использовать?__ <br>
_На мой взгляд в данном случае лучше всего подойдут money precision@k (так как будет учтена денежная составляющая, которая довольно важна в этой задаче) и MAP@k (так как учитывается наличие большого количества пользователей и важность ранжирования)._ <br><br>
- __Сколько раз в неделю отпрпавляем рассылку?__ <br>
_Нужны консультации с людьми, отвечающими за коммуникации с клиентами, чтобы понимать каким образом выстроено общение с клиентами. На мой взгляд, лучше не отправлять подобную рассылку чаще раза в неделю, чтобы не спровоцировать отказ клиентов от рассылки совсем._ <br><br>
- __В какое время отправляем рассылку?__ <br>
_Аналогично предыдущему пункту, нужны дополнительные консультации со специалистами в данной области. Зависит среди прочего от ряда факторов: когда пользователи, которых мы таргетируем, совершают обычно покупки в нашем магазине (чтобы отправить рассылку раньше момента покупки и клиент был проинформирован о предложениях), когда происходит доставка товара в магазин и сколько времени занимает разгрузка и размещение на полках (чтобы товар был в наличии, когда клиент придет осуществлять покупку) и т.д._ <br><br>
- __Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались?__ <br>
_Как уже говорилось ранее добиться разнообразия можно включив разные категории товаров в список рекомендаций: акционные, сезонные и т.д. Они будут автоматически меняться с течением времени. Стоит заметить, что так как в данном случае мы работаем с продуктовым ритейлом, то у нас будет стабильный поток новых данных, которые можно учитывать для донастройки рекомендаций. Особенно, если делать рассылку не чаще раза в неделю._ <br><br>
- __Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?__ <br>
_Этот вопрос связан с предыдущим. На мой взгляд, очевидно среди рекомендуемых должны быть товары разных категорий, а не одной, но разных брендов. Во-первых это будет лучше выглядеть, а во-вторых увеличит шансы на то, что пользователь решится на покупку как минимум одного из них. Добиться этого можно, учитывая не только вероятность или ожидаемую маржинальность, но и категорию товара при создании списка рекомендаций._ <br><br>
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?) <br>
_Может быть смотреть как соотносится текущая цена с прошлой/прошлой средней ценой. И если она меньше, то рассматривать это как акцию?_ <br><br>
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель?) <br>
_Для таких товаров тоже нужно рассчитывать вероятность покупки пользователем или ожидаемую маржинальность товара для пользователя. Можно смотреть на покупки похожих пользователей._ <br><br>
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже?) <br>
_Можно рассчитать стоимость покупаемых товаров в разрезе категории и предлагать какой-то товар из этой категории, но немного дороже (лучше, на мой взгляд, все же ориентироваться на маржинальность). На сколько дороже зависит от разных факторов: от абсолютной цены товара, от среднего чека покупателя (его покупательной способности), инфляции и т.д._

__Задание 4.__ Поэкспериментировать с ALS (grid-search)

Скопируем необходимые преобразования из лекции.

In [99]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [100]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [101]:
# Заведем фиктивный item_id

data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix)

user_item_matrix.head(3)

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

Будем использовать модель с BM25 взвешиванием, так как она показала самые лучшие результаты на лекции.

In [103]:
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [104]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

In [105]:
factors = [32, 64, 96, 128]
regularization = [0.01, 0.05, 0.1]
iterations = [10, 15, 30]

In [106]:
%%time

for factor in factors:
    for reg in regularization:
        for itr in iterations:
            model = AlternatingLeastSquares(factors=factor, 
                                            regularization=reg,
                                            iterations=itr, 
                                            calculate_training_loss=True, 
                                            num_threads=10,
                                            use_gpu=False)

            model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=False)
            
            result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
            metric_value = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
            print(f'factors={factor}, regularization={reg}, iterations={itr}: {metric_value:.6f}.')

factors=32, regularization=0.01, iterations=10: 0.185535.
factors=32, regularization=0.01, iterations=15: 0.181919.
factors=32, regularization=0.01, iterations=30: 0.183425.
factors=32, regularization=0.05, iterations=10: 0.187343.
factors=32, regularization=0.05, iterations=15: 0.184229.
factors=32, regularization=0.05, iterations=30: 0.183727.
factors=32, regularization=0.1, iterations=10: 0.186740.
factors=32, regularization=0.1, iterations=15: 0.185234.
factors=32, regularization=0.1, iterations=30: 0.182923.
factors=64, regularization=0.01, iterations=10: 0.206429.
factors=64, regularization=0.01, iterations=15: 0.202913.
factors=64, regularization=0.01, iterations=30: 0.199799.
factors=64, regularization=0.05, iterations=10: 0.204119.
factors=64, regularization=0.05, iterations=15: 0.203817.
factors=64, regularization=0.05, iterations=30: 0.201909.
factors=64, regularization=0.1, iterations=10: 0.205726.
factors=64, regularization=0.1, iterations=15: 0.205424.
factors=64, regular

Видно, что наилучшее значение метрики - __0.224108__ достигается, когда количество факторов равно 128, коэффициент регуляризации - 0.05 и количество итераций равно 10.

Попробуем добавить взвешивание в модель и рассчитать ее для найденных с помощью поиска по сетки параметров.

In [123]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )
user_item_matrix = user_item_matrix.astype(float)
user_item_matrix.tail(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
2498,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2499,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


В качестве весов используем доля продаж товара в общем объеме продаж (исключив при этом продажи товаров, которые не входят в топ 5000 популярных).

In [124]:
sales = data_train.groupby('item_id')['sales_value'].sum().reset_index()
sales.rename(columns={'sales_value': 'sold'}, inplace=True)

total_sum = np.sum(sales['sold']) - sales.loc[sales['item_id'] == 999999, 'sold'].item()

sales['weight'] = sales['sold'] / total_sum
sales.head()

,item_id,sold,weight
0,202291,81.59,0.000018
1,397896,3001.81,0.000680
2,420647,463.81,0.000105
3,480014,960.72,0.000218
4,545926,58.37,0.000013


Умножим элементы матрицы user_item на веса и дальше оценим качество модели.

In [125]:
for column in user_item_matrix.columns.to_list():
    user_item_matrix.loc[:, column] *= sales.loc[sales['item_id'] == column, 'weight'].item()
    
user_item_matrix.tail(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
2498,0.000037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000072,0.0,0.0,0.0,0.0,0.0,0.0
2499,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2500,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
sparse_user_item = csr_matrix(user_item_matrix)
user_item_matrix = bm25_weight(user_item_matrix.T).T

In [127]:
model = AlternatingLeastSquares(factors=128, 
                                regularization=0.05,
                                iterations=10, 
                                calculate_training_loss=True, 
                                num_threads=10,
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
                      show_progress=False)
            
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

0.3176293319939729

Как видно, благодаря взвешиванию удалось добиться роста значения метрики.